March Madness 

In [2]:
import pandas as pd

In [3]:
historical_data = pd.read_csv('cbb.csv')
historical_data.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017


In [ ]:
current_szn = pd.read_csv('2024-25 College Basketball.csv')
current_szn.head()

,Unnamed: 0,Unnamed: 1,Overall,Overall.1,Overall.2,Overall.3,Overall.4,Overall.5,Unnamed: 8,Conf.,...,Totals.7,Totals.8,Totals.9,Totals.10,Totals.11,Totals.12,Totals.13,Totals.14,Totals.15,Totals.16
0,Rk,School,G,W,L,W-L%,SRS,SOS,NaN,W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
1,1,Abilene Christian,27,14,13,0.519,-5.2,-1.95,NaN,6,...,412,582,0.708,289,898,347,292,73,391,553
2,2,Air Force,28,4,24,0.143,-7.34,2.73,NaN,1,...,327,512,0.639,210,835,371,160,82,374,494
3,3,Akron,28,22,6,0.786,3.07,-4.05,NaN,14,...,333,446,0.747,333,1110,514,210,104,347,515
4,4,Alabama,28,23,5,0.821,26.25,14.86,NaN,12,...,530,734,0.722,375,1224,476,168,129,369,528
